In [1]:
import pandas as pd
import numpy as np

In [2]:
address_url = "dataset/hi.csv"
address = pd.read_csv(address_url)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = address[:1000]
df = df.drop(['NUMBER','STREET', 'UNIT','CITY', 'DISTRICT', 'REGION', 'POSTCODE', 'ID', 'HASH'], axis=1)

In [4]:
import hdbscan
X = np.array(df)
X= X.astype(np.float)
rads = np.radians(X)

kms_per_radian = 6371.0088
epsilon = 1.2 / kms_per_radian
clusterer = hdbscan.HDBSCAN(min_cluster_size=10, metric='haversine')
cluster_labels = clusterer.fit_predict(df)
df['Cluster']=cluster_labels

cluster_count = df.groupby('Cluster').count().sort_values(by=['LAT'],ascending=False)[:10]
cluster_count = cluster_count.drop(['LON'],axis=1)
x = cluster_count.LAT.keys()
cluster_count
num_clusters = len(set(cluster_labels))
clusters = pd.Series([rads[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))

from geopy.distance import great_circle
from shapely.geometry import MultiPoint
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

#x = np.ndarray()
rep = pd.DataFrame(columns=['LON', 'LAT', 'Count']);
    #print(clusters[0])
for i in clusters[:len(clusters)-1]:
    length = len(i)
    x =  get_centermost_point(i)
    #print(x)
    rep = rep.append({'LON':x[1], 'LAT': x[0], 'Count':length},ignore_index=True)
#print(rep)
rep = rep.sort_values(by='Count', ascending=False)[:10]

print(rep)
    
#centermost_points = clusters.map(get_centermost_point)

/home/abhi/Desktop/scikit-learn/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


Number of clusters: 28
         LON       LAT  Count
6   0.373703 -2.754164   79.0
26  0.371500 -2.754154   57.0
19  0.373541 -2.756876   53.0
4   0.374217 -2.760811   52.0
15  0.373413 -2.757798   51.0
16  0.372082 -2.757861   43.0
9   0.373523 -2.753139   42.0
17  0.373152 -2.756339   32.0
14  0.372460 -2.755793   30.0
20  0.371519 -2.752642   28.0


In [9]:
kms_per_radian = 6371.0088
epsilon = 1.2 / kms_per_radian
clusterer = hdbscan.HDBSCAN(min_cluster_size=10, metric='haversine')
cluster_labels = clusterer.fit_predict(df)

In [12]:
df['Cluster']=cluster_labels

In [13]:
cluster_count = df.groupby('Cluster').count().sort_values(by=['LAT'],ascending=False)[:10]
cluster_count = cluster_count.drop(['LON'],axis=1)
x = cluster_count.LAT.keys()
cluster_count
num_clusters = len(set(cluster_labels))
clusters = pd.Series([rads[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))

Number of clusters: 28


In [14]:
print(cluster_count)

         LAT
Cluster     
-1       277
 6        79
 26       57
 19       53
 4        52
 15       51
 16       43
 9        42
 17       32
 14       30


In [17]:
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

#x = np.ndarray()
rep = pd.DataFrame(columns=['LON', 'LAT', 'Count']);
    #print(clusters[0])
for i in clusters[:len(clusters)-1]:
    length = len(i)
    x =  get_centermost_point(i)
    #print(x)
    rep = rep.append({'LON':x[1], 'LAT': x[0], 'Count':length},ignore_index=True)
#print(rep)
rep = rep.sort_values(by='Count', ascending=False)[:10]

print(rep)
    
#centermost_points = clusters.map(get_centermost_point)

         LON       LAT  Count
6   0.373703 -2.754164   79.0
26  0.371500 -2.754154   57.0
19  0.373541 -2.756876   53.0
4   0.374217 -2.760811   52.0
15  0.373413 -2.757798   51.0
16  0.372082 -2.757861   43.0
9   0.373523 -2.753139   42.0
17  0.373152 -2.756339   32.0
14  0.372460 -2.755793   30.0
20  0.371519 -2.752642   28.0


In [ ]:
#a = np.degrees